In [10]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

### Load csv files
Further below in the notebooks, there is code to go from class_overview.csv to folder_paths.csv, and in turn from folder_paths.csv to txtfile_paths.csv


In [15]:
import pandas as pd
import os 

overview_df = pd.read_csv("../data/class_overview.csv")
folder_paths_df = pd.read_csv("../data/folder_paths.csv")
txtfile_paths = pd.read_csv("../data/txtfile_paths.csv")
# txtfile_paths = txtfile_paths.head()
display(txtfile_paths)

,label,path,id,set
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test
...,...,...,...,...
33123,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33123,train
33124,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33124,val
33125,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33125,train
33126,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33126,train


### Load txt file, for either training, test, validation or complete dataset
Takes as input a df where each row contains the path to the document and the label of the doc. Below there is code to get a all the file paths, from the folder paths. 

In [16]:
# load txt files of the paths in 'input_df'
# choose to only load part of the data -> 'load'

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import pandas as pd
from PyPDF2 import PdfReader
import numpy as np

def count_pages(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            return len(reader.pages)
    except Exception as e:
        print(f"Error counting pages for '{pdf_path}': {e}")
        return np.nan

def clean_tokens(tokens):
    stop_words = set(stopwords.words('dutch'))
    tokens_without_stopwords = [word for word in tokens if word.lower() not in stop_words]
    tokens_without_punctuation = [word for word in tokens_without_stopwords if word not in string.punctuation and len(word)>1]
    return tokens_without_punctuation

def load_txt_files(input_df, load):
    # load is a list containing which part of the data to load, ['train', 'test', 'val', 'complete']
    df = input_df.copy()

    # create empty dataframe
    columns_list = list(df.columns.values)
    columns_list.extend(['text', 'tokens', 'token_count','clean_tokens','clean_tokens_count','pdf_path', 'num_pages'])
    return_df = pd.DataFrame(columns=columns_list)

    # if load is complete dataset, load all txt files
    if load==['complete']:
        subdf = df.copy()

     # if load is subsections of the data - examples: ['train', 'test'], ['train'], ['test'], ['train', 'val']
    else: 
        subdf = df.loc[df['set'].isin(load)]

    for index, row in subdf.iterrows():
        # extract text
        with open(row['path']) as txt_file:
            text = txt_file.read()

        # check if text is longer than 5 characters
        if len(text) > 5:
            tokens = word_tokenize(text)
            len_tokens = len(tokens)

            # clean tokens
            cleaned_tokens = clean_tokens(tokens)
            len_cleaned_tokens = len(cleaned_tokens)

            # count pages
            pdf_path = row['path'].replace('.ocr', '')
            num_pages = count_pages(pdf_path)

            # save in dataframe
            return_df.loc[len(return_df)] = {'label':row['label'], 'path':row['path'], 'id':row['id'],'set':row['set'],'text': text, 'tokens':tokens, 'token_count':len_tokens, 'clean_tokens':cleaned_tokens, "clean_tokens_count":len_cleaned_tokens,'pdf_path':pdf_path, 'num_pages':num_pages}

    return return_df

txt_files_df = load_txt_files(txtfile_paths,  ['complete'])
display(txt_files_df)

# save to blobfuse
txt_files_df.to_pickle(f"{cf.output_path}/txtfiles.pkl")

Error counting pages for '/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2013/1/https:||amsterdam.raadsinformatie.pdf': [Errno 2] No such file or directory: '/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2013/1/https:||amsterdam.raadsinformatie.pdf'
Error counting pages for '/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2013/10/https:||amsterdam.raadsinformatie.pdf': [Errno 2] No such file or directory: '/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2013/10/https:||amsterdam.raadsinformatie.pdf'


incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrec

Error counting pages for '/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/search_results/amendement/2275509.pdf': startxref not found


incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
Bad pipe message: %s [b'3\xdd \xb6\n\x91\xa3\xdb\x94s\x1b\x97\xae\x86@\x81t\xc1 \xf5\xcf\xbdn\x17b2\xed\xecv\xec\x19_\xc8\x84\xden\xf7z_!^\xd7\x073\x96\x03\xacC\xd68\x04\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",389,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",192,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",464,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",225,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",261,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",134,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0
...,...,...,...,...,...,...,...,...,...,...,...
33112,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33123,train,"In Amsterdam is, net als in andere delen van h...","[In, Amsterdam, is, ,, net, als, in, andere, d...",3816,"[Amsterdam, net, delen, land, sprake, tekort, ...",2001,/home/azureuser/cloudfiles/code/blobfuse/raads...,10.0
33113,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33124,val,| September 2021 C 1 / )\nr 1E | rs | a 4 in\n...,"[|, September, 2021, C, 1, /, ), r, 1E, |, rs,...",85228,"[September, 2021, 1E, rs, ke, We, ear, nrc, TW...",43467,/home/azureuser/cloudfiles/code/blobfuse/raads...,164.0
33114,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33125,train,WPI KWARTAAL FACTSHEET DECEMBER 2022\nKERNCIJF...,"[WPI, KWARTAAL, FACTSHEET, DECEMBER, 2022, KER...",1595,"[WPI, KWARTAAL, FACTSHEET, DECEMBER, 2022, KER...",970,/home/azureuser/cloudfiles/code/blobfuse/raads...,7.0
33115,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33126,train,"WPI VOORTGANGSRAPPORTAGE\nKERNCIJFERS WERK, PA...","[WPI, VOORTGANGSRAPPORTAGE, KERNCIJFERS, WERK,...",1821,"[WPI, VOORTGANGSRAPPORTAGE, KERNCIJFERS, WERK,...",1024,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0


In [18]:
# load the txtfiles.csv (that we just saved)
import ast
import pandas as pd

txtfiles = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
# txtfiles['tokens'] = txtfiles['tokens'].apply(ast.literal_eval)

display(txtfiles)

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",389,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",192,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",464,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",225,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",261,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",134,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0
...,...,...,...,...,...,...,...,...,...,...,...
33112,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33123,train,"In Amsterdam is, net als in andere delen van h...","[In, Amsterdam, is, ,, net, als, in, andere, d...",3816,"[Amsterdam, net, delen, land, sprake, tekort, ...",2001,/home/azureuser/cloudfiles/code/blobfuse/raads...,10.0
33113,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33124,val,| September 2021 C 1 / )\nr 1E | rs | a 4 in\n...,"[|, September, 2021, C, 1, /, ), r, 1E, |, rs,...",85228,"[September, 2021, 1E, rs, ke, We, ear, nrc, TW...",43467,/home/azureuser/cloudfiles/code/blobfuse/raads...,164.0
33114,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33125,train,WPI KWARTAAL FACTSHEET DECEMBER 2022\nKERNCIJF...,"[WPI, KWARTAAL, FACTSHEET, DECEMBER, 2022, KER...",1595,"[WPI, KWARTAAL, FACTSHEET, DECEMBER, 2022, KER...",970,/home/azureuser/cloudfiles/code/blobfuse/raads...,7.0
33115,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33126,train,"WPI VOORTGANGSRAPPORTAGE\nKERNCIJFERS WERK, PA...","[WPI, VOORTGANGSRAPPORTAGE, KERNCIJFERS, WERK,...",1821,"[WPI, VOORTGANGSRAPPORTAGE, KERNCIJFERS, WERK,...",1024,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0


# Get csv file with all unique paths/txt files
Starting point: csv file with folder paths to either files or other folder, usually one path for each class
First step: get all folder_paths that leads to txt files, some paths in the starting point are nested (for example, month and year in Motie)
Second step: get all txt_paths, and split the paths into test, training, set

### Functions to create folder_paths.csv

In [ ]:
# specifically adjusted for Moties and Onderzoeksrapporten
# Goal: add row for each path, so that each path leads to files, and not folders
def get_all_paths_from_nested_folders(input_df):
    df = input_df.copy()
    folder_paths = df.loc[df['path_end']=='FOLDERS']
    print(len(folder_paths))
    files_paths = df.loc[df['path_end']=='FILES']
    print(len(files_paths))

    moties_counter = 0
    counter = 0
    # get all paths for folder paths that lead into folder, instead of paths (this is the case for Moties and Onderzoeksrapporten)
    for index, row in folder_paths.iterrows():
        path = row['Path']
        folders = os.listdir(f"{cf.blobfuse_path}{path}")

        if row['Path'].endswith('Moties'):
            for year in folders:
                months = os.listdir(f"{cf.blobfuse_path}{path}/{year}")
                for month in months:
                    full_path = f"{cf.blobfuse_path}{path}/{year}/{month}"
                    moties_counter+=1
                    df.loc[len(df)] = {'Category':row['Category'], 'Scraped docs': 'missing', 'Status': row['Status'], 'Converted':row['Converted'], 'Path':full_path, 'path_end':'FULL_PATH'}

        else:
            for year in folders:
                full_path = f"{cf.blobfuse_path}{path}/{year}"
                counter +=1
                df.loc[len(df)] = {'Category':row['Category'], 'Scraped docs': 'missing', 'Status': row['Status'], 'Converted':row['Converted'], 'Path':full_path, 'path_end':'FULL_PATH'}

    # get all paths from folder that lead into files
    for index, row in files_paths.iterrows():
        full_path = f"{cf.blobfuse_path}{row['Path']}"
        df.loc[len(df)] = {'Category':row['Category'], 'Scraped docs': 'missing', 'Status': row['Status'], 'Converted':row['Converted'], 'Path':full_path, 'path_end':'FULL_PATH'}

    print(moties_counter, counter)
    return df

# new_df = get_all_paths_from_nested_folders(overview_df)
# new_df.to_csv("../data/folder_paths.csv", index=False)

In [36]:
# Adds column to df with information about the path, if the path leads to folder or column
def check_path_if_endswith_folder_or_file(df):
    folder_or_file = []
    for path in df['Path']:
        if os.listdir(f"{cf.blobfuse_path}{path}")[0].lower().endswith('.pdf'):
            print(f"FILES: {path}")
            folder_or_file.append('FILES')

        else:
            print(f"FOLDERS: {path}")            
            folder_or_file.append('FOLDERS')

    df['path_end'] = folder_or_file
    return df


### Functions to create txtfile_paths.csv

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd

def split_test_train_val(input_df):
    df = input_df.copy()

    X = df.drop(columns=['label'])
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

    df['set'] = 'train' 
    df.loc[X_test.index, 'set'] = 'test'  
    df.loc[X_val.index, 'set'] = 'val' 

    return df

In [48]:
import pandas as pd
import os 


# get all txtfile paths and label them with the class
# add ID and split data into test, train and val

def get_txt_files_path():
    # load dataframe with paths to classes
    overview = pd.read_csv("../data/folder_paths.csv")

    # seperate paths that lead straight to files and paths that lead to folders
    files_paths = overview.loc[overview['path_end']=='FULL_PATH']
    
    # create dataframe to store data, each row corresponds to one txt file
    df = pd.DataFrame(columns=['label', 'path', 'id'])

    id_counter = 0

    # for each path get the txt files
    for index, row in files_paths.iterrows():

        # get all text files in folder
        path = row['Path']
        file_names = os.listdir(f"{path}")
        txt_files = [filename for filename in file_names if filename.endswith('.ocr')]

        # for each txt file extract text
        for file in txt_files:
            
            # path to text file
            file_path = f'{path}/{file}'

            # create row for file
            df.loc[len(df)] = {'label':row['Category'], 'path':file_path, 'id':id_counter}
            id_counter+=1

    df = split_test_train_val(df)
    return df

txt_df = get_txt_files_path()
display(txt_df)
txt_df.to_csv("../data/txtfile_paths.csv", index=False)


,label,path,id,set
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test
...,...,...,...,...
33123,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33123,train
33124,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33124,val
33125,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33125,train
33126,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33126,train


### Gibberish code - Goal: get overview of how many documents per category

In [ ]:
import pandas as pd
import os 


# function to extract all txt files from all the folder where the categories are located
# All paths in all_paths_to_categories.csv where path_end==FILES, will the OCR files be extracted

def load_txt_files():
    # load dataframe with paths to classes
    overview = pd.read_csv("../data/folder_paths.csv")

    # seperate paths that lead straight to files and paths that lead to folders
    files_paths = overview.loc[overview['path_end']=='FULL_PATH']
    
    # create dataframe to store data, each row corresponds to one txt file
    df = pd.DataFrame(columns=['label', 'text', 'path'])

    # for each path get the txt files
    for index, row in files_paths.iterrows():

        # get all text files in folder
        path = row['Path']
        file_names = os.listdir(f"{path}")
        txt_files = [filename for filename in file_names if filename.endswith('.ocr')]

        print(row['Category'], len(txt_files), len(overview.loc[overview['Category']==row['Category']]))

        # for each txt file extract text
        for file in txt_files:
            
            # path to text file
            file_path = f'{path}/{file}'

            # extract text
            with open(file_path) as txt_file:
                text = txt_file.read()

            # create row for file
            df.loc[len(df)] = {'label':row['Category'], 'text': text, 'path':file_path}

 


    return df

txt_df = load_txt_files()
display(txt_df)


In [16]:
import os

# Start from folder that has one class
def count_doc(folder_path):
    counter = 0
    for year in os.listdir(folder_path):
        if year != '.keep':
            for month in os.listdir(f'{folder_path}/{year}'):
                counter += len(os.listdir(f'{folder_path}/{year}/{month}'))

    return counter

count_doc(f'/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/')

/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2017-gemeenteraad
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2018-gemeenteraad
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2019-gemeenteraad
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2020-gemeenteraad
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2021-gemeenteraad
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2022-gemeenteraad
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch//onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2023-gemeenteraad


3920

In [4]:
ls /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/

BerichtenCollege/  Moties/               annotated/
IngekomenStukken/  SchriftelijkeVragen/  search_results/


In [11]:
ls /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2017-gemeenteraad/

1ste-bestuursrapportage-maatschappelijke-ondersteuning.pdf*
1ste-bestuursrapportage-maatschappelijke-ondersteuning.pdf.ocr*
1ste-bestuursrapportage-maatschappelijke-ondersteuning.pdf.txt*
2-meting-pilot-rembrandtplein.pdf*
2-meting-pilot-rembrandtplein.pdf.ocr*
2-meting-pilot-rembrandtplein.pdf.txt*
aanpak---eenzaamheid-pak-je-samen-aan.pdf*
aanpak---eenzaamheid-pak-je-samen-aan.pdf.ocr*
aanpak---eenzaamheid-pak-je-samen-aan.pdf.txt*
aanpak-gezond-gewicht-2016-uitvoeringsplan-2017.pdf*
aanpak-gezond-gewicht-2016-uitvoeringsplan-2017.pdf.ocr*
aanpak-gezond-gewicht-2016-uitvoeringsplan-2017.pdf.txt*
aanpak-gezond-gewicht-2017-outcome-monitor.pdf*
aanpak-gezond-gewicht-2017-outcome-monitor.pdf.ocr*
aanpak-gezond-gewicht-2017-outcome-monitor.pdf.txt*
aanpak-op-maat-voor-eritrese-statushouders.pdf*
aanpak-op-maat-voor-eritrese-statushouders.pdf.ocr*
aanpak-op-maat-voor-eritrese-statushouders.pdf.txt*
aanpak-radicalisering-en-polarisatie.pdf*
aanpak-radicalisering-en-polarisatie.pdf.ocr*
aan

In [5]:
print(len(os.listdir(f'{cf.folder_path_raadsinformatie}search_results/voordrachten')))

3993


In [58]:
counter = 0
for i in os.listdir(f'{cf.folder_path_raadsinformatie}search_results/woo_queries'):
    for k in os.listdir(f'{cf.folder_path_raadsinformatie}search_results/woo_queries/{i}'):
        if k.lower().endswith('.pdf'):
            counter += 1

print(counter)

14054


In [41]:
1331*3

3993

In [56]:
ls /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/search_results/woo_queries/fatbikes

10005720.pdf*       12009978.pdf*       13630846.pdf.txt*   7517305.pdf.ocr*
10005720.pdf.ocr*   12009978.pdf.ocr*   13630846.pdf.xlsx*  7517305.pdf.txt*
10005720.pdf.txt*   12009978.pdf.txt*   13650589.pdf*       7517306.pdf*
10016822.pdf*       12009979.pdf*       13650589.pdf.ocr*   7517306.pdf.ocr*
10016822.pdf.ocr*   12009979.pdf.ocr*   13650589.pdf.txt*   7517306.pdf.txt*
10016822.pdf.txt*   12009979.pdf.txt*   13652616.pdf*       7517308.pdf*
10035731.pdf*       12009982.pdf*       13652616.pdf.ocr*   7517308.pdf.ocr*
10035731.pdf.ocr*   12009982.pdf.ocr*   13652616.pdf.txt*   7517308.pdf.txt*
10035731.pdf.txt*   12009982.pdf.txt*   13652694.pdf*       7517309.pdf*
10067909.pdf*       12011736.pdf*       13652694.pdf.ocr*   7517309.pdf.ocr*
10067909.pdf.ocr*   12011736.pdf.ocr*   13652694.pdf.txt*   7517309.pdf.txt*
10067909.pdf.txt*   12011736.pdf.txt*   13652695.pdf*       7526458.pdf*
10100763.pdf*       12012831.pdf*       13652705.pdf*       7526458.pdf.ocr*
10100763.pdf.oc

In [20]:
ls /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/annotated/Advies

'A - 03 Democratisering (inclusief Bestuurlijk stelsel) (17) bijlage 1 Advies.pdf'*
'A - 04 Ruimtelijke Ordening (23) 4a_ Advies DB Oost concept zienswijze ambit.pdf'*
'A - 04 Ruimtelijke Ordening (23) 4b_ Advies commissie stadsgebied Weesp conc.pdf'*
'A - 04 Ruimtelijke Ordening (23) 4c_ Advies DB Zuid concept zienswijze ambit.pdf'*
'A - 04 Ruimtelijke Ordening (23) 4d_ Advies DB Zuidoost concept zienswijze A.pdf'*
'A - 05 Ruimtelijke Ordening (23) 2_ DB-besluit- advies DB over BP Kostverlor.pdf'*
'A - 11 Duurzaamheid, Energietransitie en Circulaire Economie (42) 1_ Advies.pdf'*
'A - 14 Sport en Bewegen (28) Advies en bestuurlijke reactie inzake Zwembad I.pdf'*
'A - 16 Ruimtelijke Ordening (23) 2_ DEF TAC advies conceptontwerp BP Nieuwe.pdf'*


In [11]:
ls /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2020/1

'https:||amsterdam.raadsinformatie.nl|document|10082226|1|Afd_+motie+100_20'*
'https:||amsterdam.raadsinformatie.nl|document|11616934|1|Afd__motie_86_20'*
'https:||amsterdam.raadsinformatie.nl|document|8417559|2|50_20_Motie+C+Ernsting+c_s_+Agenda+Autoluw+betalen+naar+gebruik'*
'https:||amsterdam.raadsinformatie.nl|document|8417660|1|48_20_Motie+A+Groen+project+Zonnepanelen+op+bovengrondse+metrostations+-+rekenregel'*
'https:||amsterdam.raadsinformatie.nl|document|8417682|1|49_20_Motie+B+van+Renssen+evaluatie+participatieproces+Klaprozenbuurt'*
'https:||amsterdam.raadsinformatie.nl|document|8417705|1|51_20_Motie+D+Ernsting+c_s_+Agenda+Autoluw+verbeteren+Kattenburgerstraat'*
'https:||amsterdam.raadsinformatie.nl|document|8417714|1|52_20_Motie+E+Ernsting+c_s_+Agenda+Autoluw+parkeerplaatsen+ook+gebundeld+opheffen'*
'https:||amsterdam.raadsinformatie.nl|document|8417726|1|53_20_Motie+F+Ernsting+c_s_+Agenda+Autoluw+Oostergangtoegang'*
'https:||amsterdam.raadsinformatie.nl|document|8417778|1|

In [13]:
import os
import random
import pandas as pd
from PyPDF2 import PdfReader
from collections import defaultdict

# Function to extract text from PDF and get page count
def extract_text_and_count_from_pdf(pdf_path):
    try: 
        text = ""
        page_count = 0
        with open(pdf_path, 'rb') as f:
            reader = PdfReader(f)
            page_count = len(reader.pages)
            for page in reader.pages:
                text += page.extract_text()
        return text, page_count
    
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return '', 0


def create_df_from_pdfs(root_folder):
    data = []
    
    # each folder in the rootfolder contains PDFs for one class
    for doc_class in os.listdir(root_folder):
        print(doc_class)

        # there was a weird folder
        if doc_class != '.keep':

            # for every PDF in a folder, save the class, extract the text and count the pages
            for pdf in  os.listdir(root_folder+doc_class)[0:1]:
                print(pdf)

                # make sure only pdf files are extracted
                # if pdf.endswith('.pdf') and len(pdf)>0:
                pdf_path = root_folder+doc_class+'/'+pdf
                pdf_text, pdf_page_count = extract_text_and_count_from_pdf(pdf_path)

                data.append({'filename':pdf, 'text':pdf_text, 'class':doc_class, 'number_pages':pdf_page_count})
    df = pd.DataFrame(data)
    return df
df = create_df_from_pdfs("/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2020/")
display(df)


1
https:||amsterdam.raadsinformatie.nl|document|10082226|1|Afd_+motie+100_20
10
11
https:||amsterdam.raadsinformatie.nl|document|10063425|2|Afd_+moties+1507+en+1513_20
12
https:||amsterdam.raadsinformatie.nl|document|10007411|2|Afd_+moties+1631+1632+1633+1635+1637+1642+1646+1670+1674+1675+en+1677_20
2
https:||amsterdam.raadsinformatie.nl|document|10063354|1|Afd_+motie+196_20
3
https:||amsterdam.raadsinformatie.nl|document|10265284|1|Afd_+motie+327_20
4
https:||amsterdam.raadsinformatie.nl|document|10007070|1|Afd_+moties+607_19+1740_19+428_20+441_20+1124_20+1260_20+en+1670_20
5
https:||amsterdam.raadsinformatie.nl|document|10063695|1|Afd_+moties+549_20+572_20+en+575_20
6
https:||amsterdam.raadsinformatie.nl|document|10306427|1|Afd_+motie+722_20
7
https:||amsterdam.raadsinformatie.nl|document|10058080|1|Afd_+motie+850_20
8
9
https:||amsterdam.raadsinformatie.nl|document|10007404|1|Afd_+moties+607_19+1740_19+428_20+441_20+1124_20+1260_20+en+1670_20


,filename,text,class,number_pages
0,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \...,1,4
1,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \n \...,11,4
2,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief\nAfdoening motie\n \n...,12,7
3,https:||amsterdam.raadsinformatie.nl|document|...,\n \n \n \n \n \n \n ...,2,1
4,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \...,3,4
5,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \n \n \n \n \n \n \n \n ...,4,6
6,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief\n \n Aan: De leden va...,5,3
7,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \n \...,6,1
8,https:||amsterdam.raadsinformatie.nl|document|...,\n Raadsinformatiebrief \n \n Aan: De l...,7,2
9,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \n \n \n \n \n \n \n \n ...,9,6


In [14]:
display(df)

,filename,text,class,number_pages
0,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \...,1,4
1,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \n \...,11,4
2,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief\nAfdoening motie\n \n...,12,7
3,https:||amsterdam.raadsinformatie.nl|document|...,\n \n \n \n \n \n \n ...,2,1
4,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \...,3,4
5,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \n \n \n \n \n \n \n \n ...,4,6
6,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief\n \n Aan: De leden va...,5,3
7,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \nAfdoening motie \n \...,6,1
8,https:||amsterdam.raadsinformatie.nl|document|...,\n Raadsinformatiebrief \n \n Aan: De l...,7,2
9,https:||amsterdam.raadsinformatie.nl|document|...,Raadsinformatiebrief \n \n \n \n \n \n \n \n ...,9,6


In [17]:
print(df.iloc[5]['text'])

Raadsinformatiebrief  
 
 
 
 
 
 
 
 
 
 
Aan:  De leden van de gemeenteraad van  Amsterdam  
Datum  23 maart  2021  
Portefeuille(s)  Sociale Zaken, Duurzaamheid, Beroepsonderwijs en toeleiding arbeidsmarkt, 
Economische  Zaken  
Portefeuillehouder(s):  Rutger Groot Wassink, Marieke van Doorninck, Simone Kukenheim, Victor 
Everhardt  
Behandeld  door  Directie Werk /  directie.secretariaat.wpi@amsterdam.nl  
Onderwerp  Meer vakmensen voor  duurzaamhei d 
Afdoening motie 607.19 van de leden Karaman en Roosma inzake Green New 
Deal, motie 1740.19 van de leden Biemond, De Heer, Bakker en Karaman 
inzake Klimaatrechtvaardigheid voor werknemers, motie 428.20 Amsterdams 
Duurzaamheids Werkgelegenheidsprogramma van de leden Roosma, El Ksahi 
en Schreuders, motie 441.20 van het lid Bakker inzake de Routekaart 
Amsterdam Klimaatneutraal 2050, motie 1124.20 van het lid Timman inzake 
Betrek het Amsterdamse onderwijs bij de Transitievisie Warmte, motie 
1260.20 van de leden Mbarki en Dantzig in